In [13]:
import keras 
import numpy as np 
import pandas as pd 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


In [14]:
## load the dataset
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";",)

data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [15]:
## Split the data into training, validation an(d test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [16]:
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [17]:
train_x = train.drop(['quality'], axis = 1).values
train_y = train[['quality']].values.ravel()

## validation dataset
test_x = test.drop(['quality'], axis = 1).values
test_y = test[['quality']].values.ravel()

## splitting this train data into train and validation
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

signature = infer_signature(train_x, train_y)

In [18]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [19]:
def objective (params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs = 3,
        train_x = train_x,
        train_y = train_y,
        valid_x = valid_x,
        valid_y = valid_y,
        test_x = test_x,
        test_y = test_y,
    )
    return result

In [20]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0,1.0)
}

In [21]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo = tpe.suggest,
        max_evals=4,
        trials = trials
    )
    
    # Fetch the details of the best run
    best_run = sorted(trials.results, key = lambda x: x["loss"])[0]
    
    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)
    
    # Print out the best parameters and corresponding loss
    print(f"Best parameters : {best}")
    print(f"best eval rmse: {best_run['loss']}")
    

2025/02/17 19:40:09 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 21s 486ms/step - loss: 35.9030 - root_mean_squared_error: 5.9919
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.0576 - root_mean_squared_error: 5.0711   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 20.4865 - root_mean_squared_error: 4.4491 - val_loss: 2.8195 - val_root_mean_squared_error: 1.6791

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 2.4973 - root_mean_squared_error: 1.5803
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.4040 - root_mean_squared_error: 1.5502 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3339 - root_mean_squared_error: 1.5272 - val_loss: 1.9951 - val_root_mean_squared_error: 1.4125

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.3495 - root_mean_squared_error: 1.1617
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7736 - root_mean_squared_error: 1.3305 
46

In [23]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 20s 456ms/step - loss: 37.6786 - root_mean_squared_error: 6.1383
30/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.8643 - root_mean_squared_error: 6.2340   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 38.6615 - root_mean_squared_error: 6.2177 - val_loss: 38.0897 - val_root_mean_squared_error: 6.1717

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 36.6817 - root_mean_squared_error: 6.0565
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.8908 - root_mean_squared_error: 6.0738 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36.9967 - root_mean_squared_error: 6.0825 - val_loss: 36.9642 - val_root_mean_squared_error: 6.0798

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 36.2871 - root_mean_squared_error: 6.0239
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.4336 - root_mean_squared_error: 6.0

In [29]:
import mlflow
logged_model = 'runs:/95b82f6e71e04c9d95065ba87d6edd7c/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[5.862057 ],
       [6.8202496],
       [6.4425383],
       ...,
       [6.100338 ],
       [6.8207827],
       [5.878961 ]], dtype=float32)

In [30]:
## Register in the model registry
mlflow.register_model(model_uri, "wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1739802740217, current_stage='None', description=None, last_updated_timestamp=1739802740217, name='wine-quality', run_id='95b82f6e71e04c9d95065ba87d6edd7c', run_link=None, source='file:///c:/Users/erand/Documents/self_studies/simple_ann_project_with_mlflow/mlruns/365984032318655699/95b82f6e71e04c9d95065ba87d6edd7c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>